In [1]:
from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster("local").setAppName("spark_sql_basic2")
sc   = SparkContext(conf=conf)

# RDD만을 이용한 데이터 추출

In [3]:

movies_rdd = sc.parallelize([
    (1, ("어벤져스", "마블")),
    (2, ("슈퍼맨", "DC")),
    (3, ("배트맨", "DC")),
    (4, ("겨울왕국", "디즈니")),
    (5, ("아이언맨", "마블"))
])


attendances_rdd = sc.parallelize([
    (1, (13934592, "KR")),
    (2, (2182227,"KR")),
    (3, (4226242, "KR")),
    (4, (10303058, "KR")),
    (5, (4300365, "KR"))
])

# 마블 영화 중 관객 수가 500만 이상인 영화를 가져오기

In [4]:
# CASE1. join 먼저, filter 나중에
movie_att = movies_rdd.join(attendances_rdd)
movie_att.filter(
    lambda x : x[1][0][1] == "마블" and x[1][1][0] > 5000000
).collect()

[(1, (('어벤져스', '마블'), (13934592, 'KR')))]

In [5]:
# CASE 2. filter 먼저, join 나중에
filtered_movies = movies_rdd.filter(lambda x : x[1][1] == '마블')
filtered_att = attendances_rdd.filter(lambda x : x[1][0] > 5000000)

filtered_movies.join(filtered_att).collect()

[(1, (('어벤져스', '마블'), (13934592, 'KR')))]

# Spark SQL 사용해 보기

In [8]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").appName("spark-sql").getOrCreate()

In [4]:
# 컬럼 추가
movies = [
    (1, "어벤져스", "마블", 2012, 4, 26),
    (2, "슈퍼맨", "DC", 2013, 6, 13),
    (3, "배트맨", "DC", 2008, 8, 6),
    (4, "겨울왕국", "디즈니", 2014, 1, 16),
    (5, "아이언맨", "마블", 2008, 4, 30)
]

In [5]:
#스키마를 알아야 한다.
movie_schema = ["id", "name", "company", "year", "month", "day"]

# 2. 데이터 프레임 만들기

In [35]:
df = spark.createDataFrame(data=movies, schema=movie_schema)

In [36]:
df.select("name").show()

+--------+
|    name|
+--------+
|어벤져스|
|  슈퍼맨|
|  배트맨|
|겨울왕국|
|아이언맨|
+--------+



In [37]:
df.filter(df.year >= 2010).show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  1|어벤져스|   마블|2012|    4| 26|
|  2|  슈퍼맨|     DC|2013|    6| 13|
|  4|겨울왕국| 디즈니|2014|    1| 16|
+---+--------+-------+----+-----+---+



In [13]:
df.createOrReplaceTempView("movies")

In [14]:
# 영화 이름만 가져오기

query = """

SELECT name
  FROM movies

"""
spark.sql(query).show()

+--------+
|    name|
+--------+
|어벤져스|
|  슈퍼맨|
|  배트맨|
|겨울왕국|
|아이언맨|
+--------+



In [ ]:
spark.stop()

In [15]:
# 2010년 이후에 개봉한 영화를 조회

query = """
SELECT *
FROM movies
WHERE YEAR > 2010
"""
spark.sql(query).show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  1|어벤져스|   마블|2012|    4| 26|
|  2|  슈퍼맨|     DC|2013|    6| 13|
|  4|겨울왕국| 디즈니|2014|    1| 16|
+---+--------+-------+----+-----+---+



In [16]:
# 2012년도 이전에 개봉한 영화의 이름과 회사를 출력
query = """
SELECT *
FROM movies
WHERE YEAR < 2012
"""
spark.sql(query).show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  3|  배트맨|     DC|2008|    8|  6|
|  5|아이언맨|   마블|2008|    4| 30|
+---+--------+-------+----+-----+---+



In [18]:
# like 문자열 데이터에서 특정 단어나 문장을 포함한 데이터를 찾을 때
# % 기호를 사용해서 문장이 매칭되는지 확인 가능!
# 제목이 ~~맨으로 끝나는 데이터의 모든 정보를 조회
query = """
SELECT *
FROM movies
WHERE NAME LIKE '%맨'
"""
spark.sql(query).show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  2|  슈퍼맨|     DC|2013|    6| 13|
|  3|  배트맨|     DC|2008|    8|  6|
|  5|아이언맨|   마블|2008|    4| 30|
+---+--------+-------+----+-----+---+



In [21]:
# BETWEEN 특정 데이터와 데이터 사이를 조회
query = """
SELECT *
FROM movies
WHERE MONTH BETWEEN 4 AND 8 
"""
spark.sql(query).show()
# 개봉 월이 4 ~ 8월 사이. 4 <= 개봉월 <= 8

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  1|어벤져스|   마블|2012|    4| 26|
|  2|  슈퍼맨|     DC|2013|    6| 13|
|  3|  배트맨|     DC|2008|    8|  6|
|  5|아이언맨|   마블|2008|    4| 30|
+---+--------+-------+----+-----+---+



In [ ]:
# Join 구현하기

In [23]:
df.show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  1|어벤져스|   마블|2012|    4| 26|
|  2|  슈퍼맨|     DC|2013|    6| 13|
|  3|  배트맨|     DC|2008|    8|  6|
|  4|겨울왕국| 디즈니|2014|    1| 16|
|  5|아이언맨|   마블|2008|    4| 30|
+---+--------+-------+----+-----+---+



테이블 하나 만들어서 조인하기

In [46]:
attendances = [
    (1, 13934592., "KR"),
    (2, 2182227.,"KR"),
    (3, 4226242., "KR"),
    (4, 10303058., "KR"),
    (5, 4300365., "KR")
]

In [47]:
# 직접 스키마 지정해 보기
from pyspark.sql.types import StringType, FloatType\
    , IntegerType\
    , StructType, StructField

In [48]:
att_schema = StructType([ # 모든 컬럼의 타입을 통칭 - 컬럼 데이터의 집합
    StructField("id", IntegerType(), True), # StructField : 컬럼
    StructField("att", FloatType(), True),
    StructField("theater_country", StringType(), True)
])

In [49]:
att_df = spark.createDataFrame(
    data=attendances,
    schema=att_schema
)

att_df.dtypes

[('id', 'int'), ('att', 'float'), ('theater_country', 'string')]

In [50]:
att_df.createOrReplaceTempView("att")

In [51]:
att_df.select('*').show()

+---+-----------+---------------+
| id|        att|theater_country|
+---+-----------+---------------+
|  1|1.3934592E7|             KR|
|  2|  2182227.0|             KR|
|  3|  4226242.0|             KR|
|  4|1.0303058E7|             KR|
|  5|  4300365.0|             KR|
+---+-----------+---------------+



movies, att

In [53]:
# df와 att_df 조인
query = '''
SELECT movies.id, movies.name, movies.company, att.att, att.theater_country
FROM movies
JOIN att ON movies.id = att.id
'''
spark.sql(query).show()

+---+--------+-------+-----------+---------------+
| id|    name|company|        att|theater_country|
+---+--------+-------+-----------+---------------+
|  1|어벤져스|   마블|1.3934592E7|             KR|
|  2|  슈퍼맨|     DC|  2182227.0|             KR|
|  3|  배트맨|     DC|  4226242.0|             KR|
|  4|겨울왕국| 디즈니|1.0303058E7|             KR|
|  5|아이언맨|   마블|  4300365.0|             KR|
+---+--------+-------+-----------+---------------+



In [54]:
spark.stop()

In [ ]:
# 데이터 프레임 API

In [ ]:
# select
df.select("*").collect()

In [ ]:
df.select("name", "company").collect()

In [ ]:
df.select(df.name, (df.year-2000).alias("year")).show()

In [ ]:
# agg : Aggreagte의 약자로써, 그룹핑 후 데이터를 하나로 합쳐주는 역할
df.agg({"id": "count"}).collect()

In [ ]:
from pyspark.sql import functions as F
df.agg(F.min(df.year)).collect()

In [ ]:
df.groupBy().avg().collect()

In [ ]:
# 회사별 개봉월의 평균
df.groupBy('company').agg({"month": "mean"}).collect()

In [ ]:
# 회사 별 월 별 영화 개수 정보


In [ ]:
# join : 다른 데이터 프레임과 사용자가 지정한 컬럼을 기준으로 합치는 작업
df.join(att_df, 'id').select(df.name, att_df.att).show()

In [ ]:
# select, where, orderBy 절 사용
marvel_df = df.select("name", "company", "year").where("company=='마블'").orderBy("id")
marvel_df.collect()

In [ ]:
spark.stop()
sc.stop()

In [ ]:
# SQL 최적화

In [55]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("trip_count_sql").getOrCreate()

In [58]:
trip_file = "learning_spark_data/fhvhv_tripdata_2020-03.csv"

In [59]:
# inferSchema : 자동으로 스키마 예측하게 하기
data = spark.read.csv(trip_file, inferSchema=True, header=True)

In [61]:
data.count()

13392904

In [62]:
data.show(5)

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0005|              B02510|2020-03-01 00:03:40|2020-03-01 00:23:39|          81|         159|   NULL|
|           HV0005|              B02510|2020-03-01 00:28:05|2020-03-01 00:38:57|         168|         119|   NULL|
|           HV0003|              B02764|2020-03-01 00:03:07|2020-03-01 00:15:04|         137|         209|      1|
|           HV0003|              B02764|2020-03-01 00:18:42|2020-03-01 00:38:42|         209|          80|   NULL|
|           HV0003|              B02764|2020-03-01 00:44:24|2020-03-01 00:58:44|         256|         226|   NULL|
+-----------------+--------------------+-------------------+-------------------+

In [63]:
data.createOrReplaceTempView("mobility_data")

In [64]:
query = """
select *
from mobility_data
limit 5
"""
spark.sql(query).show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0005|              B02510|2020-03-01 00:03:40|2020-03-01 00:23:39|          81|         159|   NULL|
|           HV0005|              B02510|2020-03-01 00:28:05|2020-03-01 00:38:57|         168|         119|   NULL|
|           HV0003|              B02764|2020-03-01 00:03:07|2020-03-01 00:15:04|         137|         209|      1|
|           HV0003|              B02764|2020-03-01 00:18:42|2020-03-01 00:38:42|         209|          80|   NULL|
|           HV0003|              B02764|2020-03-01 00:44:24|2020-03-01 00:58:44|         256|         226|   NULL|
+-----------------+--------------------+-------------------+-------------------+

In [65]:
# 스파크 SQL을 사용하는 이유

In [67]:
query = """

select split(pickup_datetime, ' ')[0] as pickup_date, count(*) as trips
from mobility_data

group by pickup_date
"""

spark.sql(query).show()

+-----------+------+
|pickup_date| trips|
+-----------+------+
| 2020-03-03|697880|
| 2020-03-02|648986|
| 2020-03-01|784246|
| 2020-03-06|872012|
| 2020-03-05|731165|
| 2020-03-04|707879|
| 2020-03-09|628940|
| 2020-03-08|731222|
| 2020-03-07|886071|
| 2020-03-10|626474|
| 2020-03-12|643257|
| 2020-03-11|628601|
| 2020-03-16|391518|
| 2020-03-13|660914|
| 2020-03-15|448125|
| 2020-03-14|569397|
| 2020-03-26|141607|
| 2020-03-25|141088|
| 2020-03-20|261900|
| 2020-03-24|141686|
+-----------+------+
only showing top 20 rows



In [68]:
# 실행 계획 살펴보기
spark.sql(query).explain(True)

== Parsed Logical Plan ==
'Aggregate ['pickup_date], ['split('pickup_datetime,  )[0] AS pickup_date#628, 'count(1) AS trips#629]
+- 'UnresolvedRelation [mobility_data], [], false

== Analyzed Logical Plan ==
pickup_date: string, trips: bigint
Aggregate [split(cast(pickup_datetime#452 as string),  , -1)[0]], [split(cast(pickup_datetime#452 as string),  , -1)[0] AS pickup_date#628, count(1) AS trips#629L]
+- SubqueryAlias mobility_data
   +- View (`mobility_data`, [hvfhs_license_num#450,dispatching_base_num#451,pickup_datetime#452,dropoff_datetime#453,PULocationID#454,DOLocationID#455,SR_Flag#456])
      +- Relation [hvfhs_license_num#450,dispatching_base_num#451,pickup_datetime#452,dropoff_datetime#453,PULocationID#454,DOLocationID#455,SR_Flag#456] csv

== Optimized Logical Plan ==
Aggregate [_groupingexpression#633], [_groupingexpression#633 AS pickup_date#628, count(1) AS trips#629L]
+- Project [split(cast(pickup_datetime#452 as string),  , -1)[0] AS _groupingexpression#633]
   +- Rel

In [69]:
# 두번째 쿼리
spark.sql("""select 
                pickup_date, 
                count(*) as trips
             from ( select
                          split(pickup_datetime, ' ')[0] as pickup_date
                          from mobility_data )
             group by pickup_date""").explain(True)

== Parsed Logical Plan ==
'Aggregate ['pickup_date], ['pickup_date, 'count(1) AS trips#637]
+- 'SubqueryAlias __auto_generated_subquery_name
   +- 'Project ['split('pickup_datetime,  )[0] AS pickup_date#636]
      +- 'UnresolvedRelation [mobility_data], [], false

== Analyzed Logical Plan ==
pickup_date: string, trips: bigint
Aggregate [pickup_date#636], [pickup_date#636, count(1) AS trips#637L]
+- SubqueryAlias __auto_generated_subquery_name
   +- Project [split(cast(pickup_datetime#452 as string),  , -1)[0] AS pickup_date#636]
      +- SubqueryAlias mobility_data
         +- View (`mobility_data`, [hvfhs_license_num#450,dispatching_base_num#451,pickup_datetime#452,dropoff_datetime#453,PULocationID#454,DOLocationID#455,SR_Flag#456])
            +- Relation [hvfhs_license_num#450,dispatching_base_num#451,pickup_datetime#452,dropoff_datetime#453,PULocationID#454,DOLocationID#455,SR_Flag#456] csv

== Optimized Logical Plan ==
Aggregate [pickup_date#636], [pickup_date#636, count(1) AS tri

In [73]:
spark.stop()

In [74]:
trip_file = "fhvhv_tripdata_2020-03.csv"
zone_file = "taxi+_zone_lookup.csv"

In [75]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("trip_count_sql").getOrCreate()

In [76]:
#운행 데이터 프레임 생성, Zone 데이터프레임 생성
trip_data = spark.read.format("csv")\
    .option("header", 'true')\
    .option('inferSchema', 'true')\
    .load('learning_spark_data/fhvhv_tripdata_2020-03.csv')
zone_data = spark.read.format("csv")\
    .option("header", 'true')\
    .option('inferSchema', 'true')\
    .load('learning_spark_data/taxi+_zone_lookup.csv')

In [77]:
trip_data.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: integer (nullable = true)



In [78]:
zone_data.printSchema()

root
 |-- LocationID: integer (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)



In [104]:
trip_data.count()

13392904

In [105]:
zone_data.count()

265

In [95]:
trip_data.show(5)

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0005|              B02510|2020-03-01 00:03:40|2020-03-01 00:23:39|          81|         159|   NULL|
|           HV0005|              B02510|2020-03-01 00:28:05|2020-03-01 00:38:57|         168|         119|   NULL|
|           HV0003|              B02764|2020-03-01 00:03:07|2020-03-01 00:15:04|         137|         209|      1|
|           HV0003|              B02764|2020-03-01 00:18:42|2020-03-01 00:38:42|         209|          80|   NULL|
|           HV0003|              B02764|2020-03-01 00:44:24|2020-03-01 00:58:44|         256|         226|   NULL|
+-----------------+--------------------+-------------------+-------------------+

In [139]:
zone_data.show(10)

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
+----------+-------------+--------------------+------------+
only showing top 10 rows



In [ ]:
# 승차 Location(PULocationID)별 개수 세기
# 하차 Location(DOLocationID)별 개수 세기
# HV0003 운송사업자의 승차 지역별 트립 건수를 집계하고, 
#가장 많은 운송사업자순으로 정렬하는 분석 쿼리  hvfhs_license_num
#운송사별 운행 건수 비교
#승차 위치 Borough별 운행 건수
#서비스 존별 승차/하차 건수

In [83]:
# 승차 Location(PULocationID)별 개수 세기
trip_data.groupBy('PULocationID').count().show(10)

+------------+------+
|PULocationID| count|
+------------+------+
|         148|116205|
|         243| 87431|
|          31|  5285|
|         137| 85552|
|          85| 46120|
|         251|  9080|
|          65| 66622|
|         255|113947|
|          53| 17571|
|         133| 27200|
+------------+------+
only showing top 10 rows



In [84]:
# 하차 Location(DOLocationID)별 개수 세기
trip_data.groupBy('DOLocationID').count().show(10)

+------------+------+
|DOLocationID| count|
+------------+------+
|         148| 91601|
|         243| 86795|
|          31|  5526|
|          85| 44509|
|         137| 80098|
|         251|  8525|
|          65| 58888|
|         255|105051|
|          53| 19013|
|         133| 27760|
+------------+------+
only showing top 10 rows



In [99]:
# HV0003 운송사업자의 승차 지역별 트립 건수를 집계하고, 
#가장 많은 운송사업자순으로 정렬하는 분석 쿼리  hvfhs_license_num
from pyspark.sql.functions import col

HV003_data = trip_data \
                    .filter(col('hvfhs_license_num') == 'HV0003') \
                    .groupBy('PULocationID') \
                    .count()
hvfhs_license_num = HV003_data.orderBy('count', ascending=False)
hvfhs_license_num.show(10)

+------------+------+
|PULocationID| count|
+------------+------+
|          61|163091|
|          76|134198|
|         132|114179|
|          79|112017|
|          37|110150|
|          42|108070|
|         138|104119|
|         244| 97324|
|          89| 95724|
|          39| 94484|
+------------+------+
only showing top 10 rows



In [91]:
#운송사별 운행 건수 비교
trip_data.groupBy('hvfhs_license_num').count().show()

+-----------------+-------+
|hvfhs_license_num|  count|
+-----------------+-------+
|           HV0004| 336606|
|           HV0005|3219535|
|           HV0003|9836763|
+-----------------+-------+



In [118]:
#승차 위치 Borough별 운행 건수
import pyspark.sql.functions as F

filter_trip_data = trip_data.filter(F.col('PULocationID').between(1,265))

join_data = zone_data.join(
    filter_trip_data,
    zone_data.LocationID == filter_trip_data.PULocationID,
    'inner'
)

join_data.groupBy('Borough').count().show(10)

+-------------+-------+
|      Borough|  count|
+-------------+-------+
|       Queens|2437383|
|          EWR|    362|
|      Unknown|    845|
|     Brooklyn|3735764|
|Staten Island| 178818|
|    Manhattan|4953140|
|        Bronx|2086592|
+-------------+-------+



In [140]:
join_data.show(10)

+----------+---------+--------------------+------------+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|LocationID|  Borough|                Zone|service_zone|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+----------+---------+--------------------+------------+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|        81|    Bronx|         Eastchester|   Boro Zone|           HV0005|              B02510|2020-03-01 00:03:40|2020-03-01 00:23:39|          81|         159|   NULL|
|       168|    Bronx|Mott Haven/Port M...|   Boro Zone|           HV0005|              B02510|2020-03-01 00:28:05|2020-03-01 00:38:57|         168|         119|   NULL|
|       137|Manhattan|            Kips Bay| Yellow Zone|           HV0003|              B02764|2020-03-01 00:03:07|2020-03-01 00:15:04|         137|  

In [119]:
#서비스 존별 승차/하차 건수
join_data.groupBy('service_zone').count().show()

+------------+-------+
|service_zone|  count|
+------------+-------+
|         EWR|    362|
|         N/A|    845|
| Yellow Zone|4025190|
|    Airports| 319610|
|   Boro Zone|9046897|
+------------+-------+



In [100]:
zone_data.show(5)

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
+----------+-------------+--------------------+------------+
only showing top 5 rows



In [103]:
trip_data.show(5)

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0005|              B02510|2020-03-01 00:03:40|2020-03-01 00:23:39|          81|         159|   NULL|
|           HV0005|              B02510|2020-03-01 00:28:05|2020-03-01 00:38:57|         168|         119|   NULL|
|           HV0003|              B02764|2020-03-01 00:03:07|2020-03-01 00:15:04|         137|         209|      1|
|           HV0003|              B02764|2020-03-01 00:18:42|2020-03-01 00:38:42|         209|          80|   NULL|
|           HV0003|              B02764|2020-03-01 00:44:24|2020-03-01 00:58:44|         256|         226|   NULL|
+-----------------+--------------------+-------------------+-------------------+

In [107]:
trip_data.select("PULocationID").distinct().orderBy("PULocationID").show(5)

+------------+
|PULocationID|
+------------+
|           1|
|           2|
|           3|
|           4|
|           5|
+------------+
only showing top 5 rows



In [123]:
zone_data.select("LocationID").count()

265

In [137]:
zone_data.groupBy("LocationID").count().show()

+----------+-----+
|LocationID|count|
+----------+-----+
|       148|    1|
|       243|    1|
|        31|    1|
|        85|    1|
|       137|    1|
|       251|    1|
|        65|    1|
|        53|    1|
|       255|    1|
|       133|    1|
|        78|    1|
|       108|    1|
|       155|    1|
|        34|    1|
|       193|    1|
|       211|    1|
|       101|    1|
|       115|    1|
|       126|    1|
|        81|    1|
+----------+-----+
only showing top 20 rows

